# Importing libraries

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, LSTM, TimeDistributed, Reshape, GRU
from tensorflow.keras.layers import MaxPool2D, MaxPooling2D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow import keras

from tensorflow.keras.callbacks import CSVLogger, EarlyStopping

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [4]:
from tensorboard.plugins.hparams import api as hp

# Loading Files

In [7]:
peeps = pd.read_csv("../preprocessing/peeps_test.csv")
peeps = peeps[~peeps.duplicated()]
peeps = peeps.set_index('timestamp')
peeps = peeps.drop(['UUID', 'temperature'], axis=1)
peeps = peeps.dropna()

In [8]:
leon = pd.read_csv("../preprocessing/leon_stationary.csv")
leon = leon[~leon.duplicated()]
leon = leon.set_index('timestamp')
leon = leon.drop(['UUID', 'temperature'], axis=1)
leon = leon.dropna()

In [9]:
guadalajara = pd.read_csv("../preprocessing/guadalajara_stationary.csv")
guadalajara = guadalajara[~guadalajara.duplicated()]
guadalajara = guadalajara.set_index('timestamp')
guadalajara = guadalajara.drop(['UUID', 'temperature'], axis=1)
guadalajara = guadalajara.dropna()

# Functions

In [10]:
def ms_temporal_set(dataset, window_size, future_target, split):
    split_percent = split
    split = int(split_percent*len(dataset))
    
    data = dataset.values
    
    def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
        data = []
        labels = []

        start_index = start_index + history_size
        if end_index is None:
            end_index = len(dataset) - target_size

        for i in range(start_index, end_index):
            indices = range(i-history_size, i, step)
            data.append(dataset[indices])

            if single_step:
                labels.append(target[i+target_size])
            else:
                labels.append(target[i:i+target_size])

        return np.array(data), np.array(labels)
    
    past_history = window_size
    future_target = future_target
    STEP=1


    X_train, y_train = multivariate_data(data, data[:, 0], 0,
                                                       split, past_history,
                                                       future_target, STEP)
    X_test, y_test = multivariate_data(data, data[:, 0],
                                                   split, None, past_history,
                                                   future_target, STEP)
    
    return X_train, y_train, X_test, y_test

# Hyper-parameter Tuning, 1 Layer

## Peeps

In [11]:
%load_ext tensorboard

In [20]:
rm -rf ./logs/

In [21]:
HP_NUM_PREV = hp.HParam('num_prev', hp.Discrete([3,6,9,12]))
HP_NUM_FUT = hp.HParam('num_future', hp.Discrete([6,9,12]))
HP_LSTM_UNITS = hp.HParam('num_lstm', hp.Discrete([1, 20, 50, 100, 200]))
HP_DENSE_UNITS = hp.HParam('num_dense', hp.Discrete([5, 10, 15, 20, 100]))


METRIC_MAPE = 'mean_absolute_percentage_error'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
    hparams=[HP_NUM_PREV, HP_NUM_FUT, HP_LSTM_UNITS,HP_DENSE_UNITS],
    metrics=[hp.Metric(METRIC_MAPE, display_name='MAPE')],
  )

In [ ]:
def two_layer_lstm(hparams, dataset):
    
    df = dataset
    future_target = hparams[HP_NUM_FUT]
    X_train, y_train, X_test, y_test = ms_temporal_set(df, hparams[HP_NUM_PREV], future_target, 0.7)

    model = Sequential()
    model.add(GRU (units = hparams[HP_LSTM_UNITS], activation='relu', input_shape = X_train.shape[-2:]))
    model.add(GRU(units = hparams[HP_LSTM_UNITS], activation='relu'))
    model.add(Dense(hparams[HP_DENSE_UNITS], activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.1)))
    model.add(Dense(future_target))

    model.compile(optimizer='adam', loss='mae')
    #print(model.summary())
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0,validation_split = (14/86), callbacks=[callback])
    
    y_pred = model.predict(X_test)
    
    y_pred = y_pred[:,future_target-1]
    y_test = y_test[:,future_target-1]

    mae = mean_absolute_error(y_test, y_pred)
        
    return mae

In [23]:
def one_layer_lstm(hparams,dataset):
    
    df = dataset
    future_target = hparams[HP_NUM_FUT]
    X_train, y_train, X_test, y_test = ms_temporal_set(df, hparams[HP_NUM_PREV], future_target, 0.7)

    model = Sequential()
    model.add(GRU (units = hparams[HP_LSTM_UNITS], activation='relu', input_shape = X_train.shape[-2:]))
    model.add(Dense(hparams[HP_DENSE_UNITS], activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.1)))
    model.add(Dense(future_target)) 

    model.compile(optimizer='adam', loss = 'mean_absolute_error')

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0,validation_split = (14/86), callbacks=[callback])

    y_pred = model.predict(X_test)
    
    y_pred = y_pred[:,future_target-1]
    y_test = y_test[:,future_target-1]

    mape = mean_absolute_percentage_error(y_test, y_pred)
        
    return mape

In [24]:
def run(run_dir, hparams, dataset):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
       
        mape = one_layer_lstm(hparams, dataset)
        tf.summary.scalar(METRIC_MAPE, mape, step=1)

In [26]:
session_num = 0

for num_prev in HP_NUM_PREV.domain.values:
    for num_fut in HP_NUM_FUT.domain.values:
        for num_lstm in HP_LSTM_UNITS.domain.values:
            for num_dense in HP_DENSE_UNITS.domain.values:
                hparams = {
                  HP_NUM_PREV: num_prev,
                  HP_NUM_FUT: num_fut,
                  HP_LSTM_UNITS: num_lstm,
                  HP_DENSE_UNITS: num_dense,  
                }
                run_name = "peeps-run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run('logs/hparam_tuning/' + run_name, hparams, peeps)
                session_num += 1

--- Starting trial: peeps-run-0
{'num_prev': 3, 'num_future': 6, 'num_lstm': 1, 'num_dense': 5}


KeyboardInterrupt: ignored

## Leon

In [28]:
rm -rf ./logs/

In [ ]:
session_num = 0

for num_prev in HP_NUM_PREV.domain.values:
    for num_fut in HP_NUM_FUT.domain.values:
        for num_lstm in HP_LSTM_UNITS.domain.values:
            for num_dense in HP_DENSE_UNITS.domain.values:
                hparams = {
                  HP_NUM_PREV: num_prev,
                  HP_NUM_FUT: num_fut,
                  HP_LSTM_UNITS: num_lstm,
                  HP_DENSE_UNITS: num_dense,  
                }
                run_name = "leon-run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run('logs/hparam_tuning/' + run_name, hparams, leon)
                session_num += 1

--- Starting trial: leon-run-0
{'num_prev': 3, 'num_future': 6, 'num_lstm': 1, 'num_dense': 5}
--- Starting trial: leon-run-1
{'num_prev': 3, 'num_future': 6, 'num_lstm': 1, 'num_dense': 10}
--- Starting trial: leon-run-2
{'num_prev': 3, 'num_future': 6, 'num_lstm': 1, 'num_dense': 15}
--- Starting trial: leon-run-3
{'num_prev': 3, 'num_future': 6, 'num_lstm': 1, 'num_dense': 20}
--- Starting trial: leon-run-4
{'num_prev': 3, 'num_future': 6, 'num_lstm': 1, 'num_dense': 100}
--- Starting trial: leon-run-5
{'num_prev': 3, 'num_future': 6, 'num_lstm': 20, 'num_dense': 5}
--- Starting trial: leon-run-6
{'num_prev': 3, 'num_future': 6, 'num_lstm': 20, 'num_dense': 10}
--- Starting trial: leon-run-7
{'num_prev': 3, 'num_future': 6, 'num_lstm': 20, 'num_dense': 15}
--- Starting trial: leon-run-8
{'num_prev': 3, 'num_future': 6, 'num_lstm': 20, 'num_dense': 20}
--- Starting trial: leon-run-9
{'num_prev': 3, 'num_future': 6, 'num_lstm': 20, 'num_dense': 100}
--- Starting trial: leon-run-10
{'n

In [ ]:
%tensorboard --logdir logs/hparam_tuning

Reusing TensorBoard on port 6006 (pid 22752), started 11:43:30 ago. (Use '!kill 22752' to kill it.)

## Guadalajara 

In [ ]:
rm -rf ./logs/

In [ ]:
session_num = 0

for num_prev in HP_NUM_PREV.domain.values:
    for num_fut in HP_NUM_FUT.domain.values:
        for num_lstm in HP_LSTM_UNITS.domain.values:
            for num_dense in HP_DENSE_UNITS.domain.values:
                hparams = {
                  HP_NUM_PREV: num_prev,
                  HP_NUM_FUT: num_fut,
                  HP_LSTM_UNITS: num_lstm,
                  HP_DENSE_UNITS: num_dense,  
                }
                run_name = "guadalajara-run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run('logs/hparam_tuning/' + run_name, hparams, guadalajara)
                session_num += 1

--- Starting trial: guadalajara-run-0
{'num_prev': 1, 'num_future': 6, 'num_lstm': 1, 'num_dense': 5}
--- Starting trial: guadalajara-run-1
{'num_prev': 1, 'num_future': 6, 'num_lstm': 1, 'num_dense': 10}
--- Starting trial: guadalajara-run-2
{'num_prev': 1, 'num_future': 6, 'num_lstm': 1, 'num_dense': 15}
--- Starting trial: guadalajara-run-3
{'num_prev': 1, 'num_future': 6, 'num_lstm': 1, 'num_dense': 20}
--- Starting trial: guadalajara-run-4
{'num_prev': 1, 'num_future': 6, 'num_lstm': 1, 'num_dense': 100}
--- Starting trial: guadalajara-run-5
{'num_prev': 1, 'num_future': 6, 'num_lstm': 20, 'num_dense': 5}
--- Starting trial: guadalajara-run-6
{'num_prev': 1, 'num_future': 6, 'num_lstm': 20, 'num_dense': 10}
--- Starting trial: guadalajara-run-7
{'num_prev': 1, 'num_future': 6, 'num_lstm': 20, 'num_dense': 15}
--- Starting trial: guadalajara-run-8
{'num_prev': 1, 'num_future': 6, 'num_lstm': 20, 'num_dense': 20}
--- Starting trial: guadalajara-run-9
{'num_prev': 1, 'num_future': 6,

--- Starting trial: guadalajara-run-78
{'num_prev': 3, 'num_future': 6, 'num_lstm': 1, 'num_dense': 20}
--- Starting trial: guadalajara-run-79
{'num_prev': 3, 'num_future': 6, 'num_lstm': 1, 'num_dense': 100}
--- Starting trial: guadalajara-run-80
{'num_prev': 3, 'num_future': 6, 'num_lstm': 20, 'num_dense': 5}
--- Starting trial: guadalajara-run-81
{'num_prev': 3, 'num_future': 6, 'num_lstm': 20, 'num_dense': 10}
--- Starting trial: guadalajara-run-82
{'num_prev': 3, 'num_future': 6, 'num_lstm': 20, 'num_dense': 15}
--- Starting trial: guadalajara-run-83
{'num_prev': 3, 'num_future': 6, 'num_lstm': 20, 'num_dense': 20}
--- Starting trial: guadalajara-run-84
{'num_prev': 3, 'num_future': 6, 'num_lstm': 20, 'num_dense': 100}
--- Starting trial: guadalajara-run-85
{'num_prev': 3, 'num_future': 6, 'num_lstm': 50, 'num_dense': 5}
--- Starting trial: guadalajara-run-86
{'num_prev': 3, 'num_future': 6, 'num_lstm': 50, 'num_dense': 10}
--- Starting trial: guadalajara-run-87
{'num_prev': 3, 'n

--- Starting trial: guadalajara-run-156
{'num_prev': 6, 'num_future': 6, 'num_lstm': 20, 'num_dense': 10}
--- Starting trial: guadalajara-run-157
{'num_prev': 6, 'num_future': 6, 'num_lstm': 20, 'num_dense': 15}
--- Starting trial: guadalajara-run-158
{'num_prev': 6, 'num_future': 6, 'num_lstm': 20, 'num_dense': 20}
--- Starting trial: guadalajara-run-159
{'num_prev': 6, 'num_future': 6, 'num_lstm': 20, 'num_dense': 100}
--- Starting trial: guadalajara-run-160
{'num_prev': 6, 'num_future': 6, 'num_lstm': 50, 'num_dense': 5}
--- Starting trial: guadalajara-run-161
{'num_prev': 6, 'num_future': 6, 'num_lstm': 50, 'num_dense': 10}
--- Starting trial: guadalajara-run-162
{'num_prev': 6, 'num_future': 6, 'num_lstm': 50, 'num_dense': 15}
--- Starting trial: guadalajara-run-163
{'num_prev': 6, 'num_future': 6, 'num_lstm': 50, 'num_dense': 20}
--- Starting trial: guadalajara-run-164
{'num_prev': 6, 'num_future': 6, 'num_lstm': 50, 'num_dense': 100}
--- Starting trial: guadalajara-run-165
{'num

--- Starting trial: guadalajara-run-233
{'num_prev': 9, 'num_future': 6, 'num_lstm': 20, 'num_dense': 20}
--- Starting trial: guadalajara-run-234
{'num_prev': 9, 'num_future': 6, 'num_lstm': 20, 'num_dense': 100}
--- Starting trial: guadalajara-run-235
{'num_prev': 9, 'num_future': 6, 'num_lstm': 50, 'num_dense': 5}
--- Starting trial: guadalajara-run-236
{'num_prev': 9, 'num_future': 6, 'num_lstm': 50, 'num_dense': 10}
--- Starting trial: guadalajara-run-237
{'num_prev': 9, 'num_future': 6, 'num_lstm': 50, 'num_dense': 15}
--- Starting trial: guadalajara-run-238
{'num_prev': 9, 'num_future': 6, 'num_lstm': 50, 'num_dense': 20}
--- Starting trial: guadalajara-run-239
{'num_prev': 9, 'num_future': 6, 'num_lstm': 50, 'num_dense': 100}
--- Starting trial: guadalajara-run-240
{'num_prev': 9, 'num_future': 6, 'num_lstm': 100, 'num_dense': 5}
--- Starting trial: guadalajara-run-241
{'num_prev': 9, 'num_future': 6, 'num_lstm': 100, 'num_dense': 10}
--- Starting trial: guadalajara-run-242
{'nu

--- Starting trial: guadalajara-run-310
{'num_prev': 12, 'num_future': 6, 'num_lstm': 50, 'num_dense': 5}
--- Starting trial: guadalajara-run-311
{'num_prev': 12, 'num_future': 6, 'num_lstm': 50, 'num_dense': 10}
--- Starting trial: guadalajara-run-312
{'num_prev': 12, 'num_future': 6, 'num_lstm': 50, 'num_dense': 15}
--- Starting trial: guadalajara-run-313
{'num_prev': 12, 'num_future': 6, 'num_lstm': 50, 'num_dense': 20}
--- Starting trial: guadalajara-run-314
{'num_prev': 12, 'num_future': 6, 'num_lstm': 50, 'num_dense': 100}
--- Starting trial: guadalajara-run-315
{'num_prev': 12, 'num_future': 6, 'num_lstm': 100, 'num_dense': 5}
--- Starting trial: guadalajara-run-316
{'num_prev': 12, 'num_future': 6, 'num_lstm': 100, 'num_dense': 10}
--- Starting trial: guadalajara-run-317
{'num_prev': 12, 'num_future': 6, 'num_lstm': 100, 'num_dense': 15}
--- Starting trial: guadalajara-run-318
{'num_prev': 12, 'num_future': 6, 'num_lstm': 100, 'num_dense': 20}
--- Starting trial: guadalajara-ru

In [ ]:
%tensorboard --logdir logs/hparam_tuning